# Delegation Unit Test
Performing verification of the delegation mechanism by comparing the event data to the results of cadCAD model.

In [45]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *

# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# print(module_path)
# sys.path.append(module_path)
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')


Configure environment

In [46]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [47]:
# print(delegation_events)
delegation_events_dict = delegation_events
print("EXPECTED TRUTH FROM DELEGATION EVENTS:")
for timestep, events in delegation_events_dict.items():
    for event in events:        
        print(f"{timestep}, {event['delegator']}, {event['shares']}")
    


EXPECTED TRUTH FROM DELEGATION EVENTS:
2, 0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, 165851.137200000000000000
3, 0xd776a7306ee6a060cebb46b46d305e88fd39ba84, 251819.644650000000000000
6, 0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, 6965.000000000000000000
7, 0x9327775e225a703125167eb423d246915bdc37e7, 9950.000000000000000000
8, 0x32345932bc863f54ef87e2ece3d458f1789b1bfc, 1991.758662500000000000
10, 0xb300749c08cab1c9ce67f66a18ea5926360ba36a, 503.905083650000000000
11, 0x57b4a5f470c85d4281f7baf5162b59243a3491df, 65257.075000000000000000
14, 0xb7797cb7b4a031c4c1e04ea14824fa5d8d30beed, 39800.000000000000000000
15, 0xf5376e847efa1ea889bfcb03706f414dade0e82c, 12488.567623993900000000
16, 0x1853bf6a892abd949aed052bb634ab96afaf31dd, 14169.714085664846265000
19, 0x1beda87a78e838687668ce2c564f1746dce7396e, 765384.845000000000000000
21, 0x7eb692f22af33d637aab31f9277653235682ebe6, 4002.893379514657642500
22, 0xb4ecbb9d116a2b38cee4ede75b0a96746a175efb, 9950.000000000000000000
23, 0x4e43a0cd79a9324

In [48]:
timestep = 1
# df.iloc[timestep-1].delegators['0xd776a7306ee6a060cebb46b46d305e88fd39ba84'].shares
# df.iloc[timestep*6+1].delegators

# timestep += 1
# df.iloc[3].delegators['0xd776a7306ee6a060cebb46b46d305e88fd39ba84']
print("MODELED RESULTS")
delegation_event_shares = {}
for timestep, events in delegation_events_dict.items():
    for event in events:
        curTimestepShares = df.iloc[timestep-1].delegators[event['delegator']].shares
        try:
            lastTimestepShares = df.iloc[timestep-2].delegators[event['delegator']].shares
        except:
            lastTimestepShares = 0
        deltaShares = curTimestepShares - lastTimestepShares
        delegation_event_shares[timestep] = deltaShares
        print(f"{timestep}, {event['delegator']}, {delegation_event_shares[timestep]}")



MODELED RESULTS
2, 0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, 165851.137200000000000000
3, 0xd776a7306ee6a060cebb46b46d305e88fd39ba84, 251819.6446500000000000000001
6, 0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, 6965.000000000000000000000003
7, 0x9327775e225a703125167eb423d246915bdc37e7, 9950.000000000000000000000003
8, 0x32345932bc863f54ef87e2ece3d458f1789b1bfc, 1991.758662500000000000000000
10, 0xb300749c08cab1c9ce67f66a18ea5926360ba36a, 503.9050836500000000000000003
11, 0x57b4a5f470c85d4281f7baf5162b59243a3491df, 65257.07500000000000000000000
14, 0xb7797cb7b4a031c4c1e04ea14824fa5d8d30beed, 39800.00000000000000000000001
15, 0xf5376e847efa1ea889bfcb03706f414dade0e82c, 12488.56762399390000000000000
16, 0x1853bf6a892abd949aed052bb634ab96afaf31dd, 14169.71408566484626500000000
19, 0x1beda87a78e838687668ce2c564f1746dce7396e, 765384.8450000000000000000000
21, 0x7eb692f22af33d637aab31f9277653235682ebe6, 4002.893379514657642500000000
22, 0xb4ecbb9d116a2b38cee4ede75b0a96746a175efb, 9950.0000

In [49]:
print("UNITTEST RESULTS")
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
for timestep, events in delegation_events_dict.items():
    for event in events:
        curTimestepShares = df.iloc[timestep-1].delegators[event['delegator']].shares
        try:
            lastTimestepShares = df.iloc[timestep-2].delegators[event['delegator']].shares
        except:
            lastTimestepShares = 0
        deltaShares = curTimestepShares - lastTimestepShares

        print(f"Event: Timestep={timestep}, Delegator ID={event['delegator']}, Shares Granted={event['shares']}")
        print(f"Model: Timestep={timestep}, Delegator ID={event['delegator']}, Shares Granted={deltaShares}")
        
        try:
            ratio = abs(event['shares'] / deltaShares)
        except:
            ratio = np.inf
        exact = ratio == 1.0
        close = 0.99 <= ratio <= 1.01
        reallyClose = 0.99999 <= ratio <= 1.00001

        if exact:            
            cntExact += 1
        elif reallyClose:
            cntReallyClose += 1
        elif close:
            cntClose += 1
        cnt += 1
        print("Shares Equal?", exact)
        print("Shares Within 0.001%?", reallyClose)
        print("Shares Within 1%?", close)
        print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Total Number: {cnt}")   
        print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Total Number: {cnt}")   


UNITTEST RESULTS
Event: Timestep=2, Delegator ID=0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, Shares Granted=165851.137200000000000000
Model: Timestep=2, Delegator ID=0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, Shares Granted=165851.137200000000000000
Shares Equal? True
Shares Within 0.001%? True
Shares Within 1%? True
Exact: 1, ReallyClose: 0, Close: 0, Total Number: 1

Event: Timestep=3, Delegator ID=0xd776a7306ee6a060cebb46b46d305e88fd39ba84, Shares Granted=251819.644650000000000000
Model: Timestep=3, Delegator ID=0xd776a7306ee6a060cebb46b46d305e88fd39ba84, Shares Granted=251819.6446500000000000000001
Shares Equal? False
Shares Within 0.001%? True
Shares Within 1%? True
Exact: 1, ReallyClose: 1, Close: 0, Total Number: 2

Event: Timestep=6, Delegator ID=0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, Shares Granted=6965.000000000000000000
Model: Timestep=6, Delegator ID=0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, Shares Granted=6965.000000000000000000000003
Shares Equal? False
Shares Wit

# Conclusion
The number of shares granted by delegation in the first two events match the output of the cadCAD model exactly and then the next 3 are extremely close, diverging gradually.